In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
    
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import numpy as np
import random
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet 
from random import sample
import string

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
data = pd.read_csv('./IMDB Dataset.csv')

In [6]:
# Create dictionary of hyper-parameters for our RNN

rnn_hyper_param = {
    'VOCAB_SIZE':50000,
    'EPOCHS':150,
    'BS':512,
    'LR':0.005,
    'OOV_TOK':"<OOV>",
    'MAX_LENGTH':520,
    'PADDING_TYPE':"post",
    'TRUNC_TYPE':"post"
}

In [7]:
# Create dictionary of hyper-parameters for our CNN

cnn_hyper_param = {
    'VOCAB_SIZE':50000,
    'EPOCHS':150,
    'BS':512,
    'LR':0.001,
    'OOV_TOK':"<OOV>",
    'MAX_LENGTH':520,
    'PADDING_TYPE':"post",
    'TRUNC_TYPE':"post"
}

In [8]:
# Keep to the same size of test set we've been using to test how well it generalizes.

x_train, x_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], 
                                                    test_size=0.2, random_state=0, 
                                                    stratify=data['sentiment'])

In [9]:
# Tokenize using the training set vocabulary only.

tokenizer = Tokenizer(num_words = rnn_hyper_param['VOCAB_SIZE'], oov_token=rnn_hyper_param['OOV_TOK'])
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

# Create sequences (i.e. apply Tokenized dictionary)
test_sequences = tokenizer.texts_to_sequences(x_test)
testing_padded = pad_sequences(test_sequences, padding=rnn_hyper_param['PADDING_TYPE'], 
                               maxlen=rnn_hyper_param['MAX_LENGTH'], truncating=rnn_hyper_param['TRUNC_TYPE'])

# Convert categorical data to numerical
y_test = list(y_test)
for i in range (len(y_test)):
    if y_test[i] == 'positive':
        y_test[i] = 1
    else:
        y_test[i] = 0
y_test = np.array(y_test)

In [10]:
# Create a new df using the train set.
data = {
    'review':x_train,
    'sentiment':y_train
}

data = pd.DataFrame(data)


In [11]:
# Iteratively split the train set into subsets with different sizes. This is used for
# testing the impact of training with lesser data with and without data augmentation

data_dict = {}
split_size = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for split in split_size:
    _, x_train, _, y_train = train_test_split(data['review'], data['sentiment'], 
                                                    test_size=split, random_state=0, 
                                                    stratify=data['sentiment'])
    df = {
        'review':x_train,
        'sentiment':y_train
    }
    
    df = pd.DataFrame(df)
    data_dict[split] = df


# <center>Data Augmentation

In [12]:
# function to create the best CNN model we found through grid search

def create_cnn():
    model = keras.Sequential([
    keras.layers.Embedding(len(word_index)+1, 30, input_length=520),
    
    keras.layers.Conv1D(40, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv1D(5, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.3),
    
    keras.layers.Flatten(),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', 
                  optimizer=keras.optimizers.Adam(learning_rate=cnn_hyper_param['LR']), 
                  metrics=['accuracy'])
    return model

In [13]:
# function to create the best RNN model we found through grid search

def create_rnn():
    model = keras.Sequential([
        keras.layers.Embedding(len(word_index)+1, 128, input_length=520),
        keras.layers.LSTM(100, return_sequences=True),
        keras.layers.Dropout(0.5),
        keras.layers.LSTM(100),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=rnn_hyper_param['LR']),
                     loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [14]:
def avgNestedLists(nested_vals):
    """
    Averages a 2-D array and returns a 1-D array of all of the columns
    averaged together, regardless of their dimensions.
    """
    
    output = []
    maximum = 0
    for lst in nested_vals:
        if len(lst) > maximum:
            maximum = len(lst)
    for index in range(maximum): # Go through each index of longest list
        temp = []
        for lst in nested_vals: # Go through each list
            if index < len(lst): # If not an index error
                temp.append(lst[index])
        output.append(np.nanmean(temp))
    return output

# <center>Best CNN without Data Augmentation

In [15]:
# Create our call backs for training our CNN. Patience values were decided by analyzing the 
# vanilla CNN behaviour during training

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4, min_lr=0.00005, verbose=1)
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=12)

In [19]:
# For each subset of data, train our CNN model on it three times.
# Store this information in a dictionary to be used later for analysis.

CNN_noaug_hist = {}
for size, dataset in data_dict.items():
    print(f"Training using {size} of dataset.")
    _history = []
    for i in range(3):
        CNN = create_cnn()
        x_train = list(dataset['review'])
        y_train = list(dataset['sentiment'])

        for j in range (len(y_train)):
            if y_train[j]=='positive':
                y_train[j] = 1
            else:
                y_train[j] = 0
        y_train = np.array(y_train)

        train_sequences = tokenizer.texts_to_sequences(x_train)
        training_padded = pad_sequences(train_sequences, padding=cnn_hyper_param['PADDING_TYPE'], 
                                        maxlen=cnn_hyper_param['MAX_LENGTH'], truncating=cnn_hyper_param['TRUNC_TYPE'])

        history = CNN.fit(training_padded, y_train, batch_size = cnn_hyper_param['BS'],
                                epochs=250, validation_data=(testing_padded, y_test),
                               callbacks=[reduce_lr, earlystop])
        _history.append(history)
    CNN_noaug_hist[f'{size}'] = _history

Training using 0.01 of dataset.
Train on 400 samples, validate on 10000 samples
Epoch 1/250
400/400 [==============================] - 1s 3ms/sample - loss: 0.6922 - accuracy: 0.5025 - val_loss: 0.6934 - val_accuracy: 0.4996
Epoch 2/250
400/400 [==============================] - 0s 1000us/sample - loss: 0.6910 - accuracy: 0.5275 - val_loss: 0.6934 - val_accuracy: 0.4993
Epoch 3/250
400/400 [==============================] - 0s 990us/sample - loss: 0.6943 - accuracy: 0.5100 - val_loss: 0.6934 - val_accuracy: 0.4986
Epoch 4/250
400/400 [==============================] - 0s 986us/sample - loss: 0.6907 - accuracy: 0.5500 - val_loss: 0.6933 - val_accuracy: 0.4981
Epoch 5/250
400/400 [==============================] - 0s 958us/sample - loss: 0.6887 - accuracy: 0.5500 - val_loss: 0.6933 - val_accuracy: 0.4976
Epoch 6/250
400/400 [==============================] - 0s 955us/sample - loss: 0.6889 - accuracy: 0.5550 - val_loss: 0.6932 - val_accuracy: 0.4984
Epoch 7/250
400/400 [==================

Epoch 29/250
400/400 [==============================] - 0s 964us/sample - loss: 0.6865 - accuracy: 0.6000 - val_loss: 0.6930 - val_accuracy: 0.5182
Epoch 30/250
400/400 [==============================] - 0s 946us/sample - loss: 0.6864 - accuracy: 0.5875 - val_loss: 0.6930 - val_accuracy: 0.5175
Epoch 31/250
400/400 [==============================] - 0s 954us/sample - loss: 0.6847 - accuracy: 0.5900 - val_loss: 0.6930 - val_accuracy: 0.5178
Epoch 32/250
400/400 [==============================] - 0s 980us/sample - loss: 0.6869 - accuracy: 0.5925 - val_loss: 0.6930 - val_accuracy: 0.5177
Epoch 33/250
400/400 [==============================] - 0s 938us/sample - loss: 0.6865 - accuracy: 0.6000 - val_loss: 0.6930 - val_accuracy: 0.5172
Epoch 34/250
400/400 [==============================] - 0s 933us/sample - loss: 0.6859 - accuracy: 0.5925 - val_loss: 0.6930 - val_accuracy: 0.5182
Epoch 35/250
400/400 [==============================] - 0s 932us/sample - loss: 0.6853 - accuracy: 0.5800 - val_

Epoch 85/250
400/400 [==============================] - 0s 962us/sample - loss: 0.6783 - accuracy: 0.6625 - val_loss: 0.6930 - val_accuracy: 0.5140
Epoch 86/250
400/400 [==============================] - 0s 916us/sample - loss: 0.6791 - accuracy: 0.6400 - val_loss: 0.6930 - val_accuracy: 0.5151
Epoch 87/250
400/400 [==============================] - 0s 930us/sample - loss: 0.6804 - accuracy: 0.6200 - val_loss: 0.6930 - val_accuracy: 0.5153
Epoch 88/250
400/400 [==============================] - 0s 944us/sample - loss: 0.6763 - accuracy: 0.6225 - val_loss: 0.6930 - val_accuracy: 0.5165
Epoch 89/250
400/400 [==============================] - 0s 928us/sample - loss: 0.6767 - accuracy: 0.6450 - val_loss: 0.6930 - val_accuracy: 0.5174
Epoch 90/250
400/400 [==============================] - 0s 913us/sample - loss: 0.6799 - accuracy: 0.6125 - val_loss: 0.6930 - val_accuracy: 0.5172
Epoch 91/250
400/400 [==============================] - 0s 923us/sample - loss: 0.6810 - accuracy: 0.5875 - val_

400/400 [==============================] - 0s 938us/sample - loss: 0.6723 - accuracy: 0.6475 - val_loss: 0.6929 - val_accuracy: 0.5150
Epoch 141/250
400/400 [==============================] - 0s 922us/sample - loss: 0.6701 - accuracy: 0.6725 - val_loss: 0.6929 - val_accuracy: 0.5152
Epoch 142/250
400/400 [==============================] - 0s 936us/sample - loss: 0.6725 - accuracy: 0.6275 - val_loss: 0.6929 - val_accuracy: 0.5143
Epoch 143/250
400/400 [==============================] - 0s 952us/sample - loss: 0.6724 - accuracy: 0.6450 - val_loss: 0.6929 - val_accuracy: 0.5144
Epoch 144/250
400/400 [==============================] - 0s 948us/sample - loss: 0.6751 - accuracy: 0.6100 - val_loss: 0.6929 - val_accuracy: 0.5146
Epoch 145/250
400/400 [==============================] - 0s 971us/sample - loss: 0.6667 - accuracy: 0.6875 - val_loss: 0.6929 - val_accuracy: 0.5143
Epoch 146/250
400/400 [==============================] - 0s 958us/sample - loss: 0.6705 - accuracy: 0.6800 - val_loss: 0

400/400 [==============================] - 0s 960us/sample - loss: 0.6607 - accuracy: 0.6850 - val_loss: 0.6927 - val_accuracy: 0.5168
Epoch 196/250
400/400 [==============================] - 0s 939us/sample - loss: 0.6635 - accuracy: 0.6500 - val_loss: 0.6927 - val_accuracy: 0.5172
Epoch 197/250
400/400 [==============================] - 0s 942us/sample - loss: 0.6580 - accuracy: 0.7225 - val_loss: 0.6927 - val_accuracy: 0.5172
Epoch 198/250
400/400 [==============================] - 0s 921us/sample - loss: 0.6514 - accuracy: 0.7400 - val_loss: 0.6927 - val_accuracy: 0.5184
Epoch 199/250
400/400 [==============================] - 0s 953us/sample - loss: 0.6578 - accuracy: 0.6725 - val_loss: 0.6927 - val_accuracy: 0.5181
Epoch 200/250
400/400 [==============================] - 0s 954us/sample - loss: 0.6553 - accuracy: 0.7350 - val_loss: 0.6927 - val_accuracy: 0.5180
Epoch 201/250
400/400 [==============================] - 0s 927us/sample - loss: 0.6559 - accuracy: 0.7125 - val_loss: 0

...

Epoch 14/250
20000/20000 [==============================] - 3s 159us/sample - loss: 0.0328 - accuracy: 0.9909 - val_loss: 0.4494 - val_accuracy: 0.8799
Epoch 15/250
20000/20000 [==============================] - 3s 165us/sample - loss: 0.0350 - accuracy: 0.9892 - val_loss: 0.4515 - val_accuracy: 0.8797
Epoch 16/250
20000/20000 [==============================] - 3s 162us/sample - loss: 0.0337 - accuracy: 0.9911 - val_loss: 0.4536 - val_accuracy: 0.8803
Epoch 17/250
20000/20000 [==============================] - 3s 159us/sample - loss: 0.0312 - accuracy: 0.9911 - val_loss: 0.4598 - val_accuracy: 0.8800
Epoch 00017: early stopping
Train on 20000 samples, validate on 10000 samples
Epoch 1/250
20000/20000 [==============================] - 4s 213us/sample - loss: 0.6931 - accuracy: 0.5074 - val_loss: 0.6931 - val_accuracy: 0.5009
Epoch 2/250
20000/20000 [==============================] - 3s 170us/sample - loss: 0.6918 - accuracy: 0.5228 - val_loss: 0.6922 - val_accuracy: 0.5320
Epoch 3/250


Epoch 10/250
24000/24000 [==============================] - 4s 157us/sample - loss: 0.0353 - accuracy: 0.9904 - val_loss: 0.4666 - val_accuracy: 0.8870
Epoch 11/250
24000/24000 [==============================] - 4s 156us/sample - loss: 0.0330 - accuracy: 0.9911 - val_loss: 0.4769 - val_accuracy: 0.8866
Epoch 12/250
23552/24000 [============================>.] - ETA: 0s - loss: 0.0288 - accuracy: 0.9927
Epoch 00012: ReduceLROnPlateau reducing learning rate to 5e-05.
24000/24000 [==============================] - 4s 157us/sample - loss: 0.0288 - accuracy: 0.9927 - val_loss: 0.4891 - val_accuracy: 0.8868
Epoch 13/250
24000/24000 [==============================] - 4s 157us/sample - loss: 0.0282 - accuracy: 0.9932 - val_loss: 0.4931 - val_accuracy: 0.8869
Epoch 14/250
24000/24000 [==============================] - 4s 157us/sample - loss: 0.0268 - accuracy: 0.9937 - val_loss: 0.4960 - val_accuracy: 0.8865
Epoch 15/250
24000/24000 [==============================] - 4s 156us/sample - loss: 0.0

27648/28000 [============================>.] - ETA: 0s - loss: 0.0530 - accuracy: 0.9847
Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
28000/28000 [==============================] - 4s 154us/sample - loss: 0.0528 - accuracy: 0.9847 - val_loss: 0.4303 - val_accuracy: 0.8899
Epoch 9/250
28000/28000 [==============================] - 4s 161us/sample - loss: 0.0377 - accuracy: 0.9890 - val_loss: 0.4320 - val_accuracy: 0.8910
Epoch 10/250
28000/28000 [==============================] - 4s 154us/sample - loss: 0.0341 - accuracy: 0.9905 - val_loss: 0.4478 - val_accuracy: 0.8884
Epoch 11/250
28000/28000 [==============================] - 4s 154us/sample - loss: 0.0320 - accuracy: 0.9915 - val_loss: 0.4620 - val_accuracy: 0.8904
Epoch 12/250
27648/28000 [============================>.] - ETA: 0s - loss: 0.0261 - accuracy: 0.9931
Epoch 00012: ReduceLROnPlateau reducing learning rate to 5e-05.
28000/28000 [==============================] - 4s 153us/sample - loss:

32000/32000 [==============================] - 5s 154us/sample - loss: 0.0997 - accuracy: 0.9678 - val_loss: 0.3157 - val_accuracy: 0.8979
Epoch 7/250
31744/32000 [============================>.] - ETA: 0s - loss: 0.0744 - accuracy: 0.9769
Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
32000/32000 [==============================] - 5s 154us/sample - loss: 0.0745 - accuracy: 0.9769 - val_loss: 0.3654 - val_accuracy: 0.8916
Epoch 8/250
32000/32000 [==============================] - 5s 153us/sample - loss: 0.0541 - accuracy: 0.9839 - val_loss: 0.3781 - val_accuracy: 0.8953
Epoch 9/250
32000/32000 [==============================] - 5s 153us/sample - loss: 0.0468 - accuracy: 0.9864 - val_loss: 0.3933 - val_accuracy: 0.8935
Epoch 10/250
32000/32000 [==============================] - 5s 159us/sample - loss: 0.0409 - accuracy: 0.9887 - val_loss: 0.4100 - val_accuracy: 0.8940
Epoch 11/250
31744/32000 [============================>.] - ETA: 0s - loss: 0.0378 - a

36000/36000 [==============================] - 6s 163us/sample - loss: 0.1576 - accuracy: 0.9462 - val_loss: 0.2787 - val_accuracy: 0.8973
Epoch 6/250
36000/36000 [==============================] - 6s 154us/sample - loss: 0.1070 - accuracy: 0.9647 - val_loss: 0.3001 - val_accuracy: 0.9007
Epoch 7/250
36000/36000 [==============================] - 6s 155us/sample - loss: 0.0775 - accuracy: 0.9763 - val_loss: 0.3797 - val_accuracy: 0.8907
Epoch 8/250
35840/36000 [============================>.] - ETA: 0s - loss: 0.0629 - accuracy: 0.9809
Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
36000/36000 [==============================] - 6s 162us/sample - loss: 0.0628 - accuracy: 0.9809 - val_loss: 0.3849 - val_accuracy: 0.8971
Epoch 9/250
36000/36000 [==============================] - 6s 155us/sample - loss: 0.0425 - accuracy: 0.9883 - val_loss: 0.4013 - val_accuracy: 0.8975
Epoch 10/250
36000/36000 [==============================] - 6s 154us/sample - loss: 0.0

In [20]:
# Average out the vaidation accuracy of all three iterations, for each subset of data we used to train
# For example, we used 1% of data to train three times. Average the validation accuracy store
# it in a dictionary. Do this for all subsets we have (0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9)

new_dict = {}
for key, item in CNN_noaug_hist.items():
    avg_list = []
    for history in item:
        print(history.history['val_accuracy'])
        avg_list.append(history.history['val_accuracy'])
    print("========================================================")
    avg_list = avgNestedLists(avg_list)
    new_dict[key] = avg_list
    print(avg_list)
    print()

[0.4996, 0.4993, 0.4986, 0.4981, 0.4976, 0.4984, 0.5011, 0.5038, 0.5028, 0.501, 0.5053, 0.5053, 0.5053, 0.5045, 0.5039, 0.5033, 0.5025, 0.5024, 0.5023, 0.5022, 0.5021, 0.5024, 0.5028, 0.5029, 0.5029]
[0.5072, 0.5034, 0.5043, 0.5133, 0.5164, 0.518, 0.5178, 0.5196, 0.5204, 0.5196, 0.5191, 0.5177, 0.5177, 0.5178, 0.518, 0.5181, 0.5185, 0.5187, 0.5189, 0.5192, 0.5189, 0.5191, 0.5189, 0.5183, 0.5186, 0.5189, 0.5185, 0.5182, 0.5182, 0.5175, 0.5178, 0.5177, 0.5172, 0.5182, 0.5184, 0.5182, 0.5187, 0.5183, 0.5186, 0.5186, 0.5198, 0.5201, 0.5197, 0.5195, 0.5189, 0.5182, 0.5175, 0.5177, 0.5178, 0.5176, 0.5176, 0.5178, 0.5176, 0.5173, 0.5172, 0.5162, 0.5153, 0.5149, 0.5152, 0.5146, 0.5129, 0.5128, 0.5126, 0.5128, 0.512, 0.5116, 0.5107, 0.5109, 0.511, 0.5107, 0.5109, 0.5118, 0.5114, 0.5122, 0.5124, 0.5127, 0.5136, 0.5131, 0.5126, 0.5135, 0.5126, 0.5126, 0.5131, 0.5133, 0.514, 0.5151, 0.5153, 0.5165, 0.5174, 0.5172, 0.5163, 0.5159, 0.5162, 0.5155, 0.5155, 0.5161, 0.5161, 0.5159, 0.5156, 0.5154, 0.51

In [25]:
# Write the results into csv files with the name being the model name and the subset size

for title, item in new_dict.items():
    temp_dict = {title:item}
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(temp_dict)

    # save to csv:
    hist_csv_file = 'CNN_NoAug_History_' + str(title) + '.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)

# <center>Best RNN without Data Augmentation

In [59]:
# Create callbacks for our RNN with patience value determined in the vanilla RNN code

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=8, min_lr=0.00005, verbose=1)
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=16)

In [60]:
# For each subset of data, train our RNN model on it three times.
# Store this information in a dictionary to be used later for analysis.

RNN_noaug_hist = {}
for size, dataset in data_dict.items():
    print(f"Training using {size} of dataset.")
    _history = []
    for i in range(3):
        RNN = create_rnn()
        x_train = list(dataset['review'])
        y_train = list(dataset['sentiment'])

        for j in range (len(y_train)):
            if y_train[j]=='positive':
                y_train[j] = 1
            else:
                y_train[j] = 0
        y_train = np.array(y_train)

        train_sequences = tokenizer.texts_to_sequences(x_train)
        training_padded = pad_sequences(train_sequences, padding=rnn_hyper_param['PADDING_TYPE'], 
                                        maxlen=rnn_hyper_param['MAX_LENGTH'], truncating=rnn_hyper_param['TRUNC_TYPE'])

        history = RNN.fit(training_padded, y_train, batch_size = rnn_hyper_param['BS'],
                                epochs=250, validation_data=(testing_padded, y_test),
                               callbacks=[reduce_lr, earlystop])
        _history.append(history)
    RNN_noaug_hist[f'{size}'] = _history

Training using 0.01 of dataset.
Train on 400 samples, validate on 10000 samples
Epoch 1/250
400/400 [==============================] - 8s 19ms/sample - loss: 0.6933 - accuracy: 0.4925 - val_loss: 0.6967 - val_accuracy: 0.4999
Epoch 2/250
400/400 [==============================] - 3s 6ms/sample - loss: 0.6955 - accuracy: 0.5050 - val_loss: 0.6951 - val_accuracy: 0.5000
Epoch 3/250
400/400 [==============================] - 3s 6ms/sample - loss: 0.6938 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/250
400/400 [==============================] - 3s 6ms/sample - loss: 0.6912 - accuracy: 0.5175 - val_loss: 0.6958 - val_accuracy: 0.5031
Epoch 5/250
400/400 [==============================] - 3s 6ms/sample - loss: 0.6952 - accuracy: 0.5350 - val_loss: 0.6934 - val_accuracy: 0.5041
Epoch 6/250
400/400 [==============================] - 3s 6ms/sample - loss: 0.6928 - accuracy: 0.5400 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 7/250
400/400 [============================

Epoch 00029: early stopping
Train on 400 samples, validate on 10000 samples
Epoch 1/250
400/400 [==============================] - 8s 19ms/sample - loss: 0.6929 - accuracy: 0.5025 - val_loss: 0.6939 - val_accuracy: 0.5000
Epoch 2/250
400/400 [==============================] - 2s 6ms/sample - loss: 0.6941 - accuracy: 0.5125 - val_loss: 0.7087 - val_accuracy: 0.4999
Epoch 3/250
400/400 [==============================] - 2s 6ms/sample - loss: 0.7086 - accuracy: 0.5150 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 4/250
400/400 [==============================] - 2s 6ms/sample - loss: 0.6937 - accuracy: 0.4950 - val_loss: 0.6959 - val_accuracy: 0.5000
Epoch 5/250
400/400 [==============================] - 2s 6ms/sample - loss: 0.6954 - accuracy: 0.5000 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 6/250
400/400 [==============================] - 2s 6ms/sample - loss: 0.6911 - accuracy: 0.5050 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 7/250
400/400 [==============================] 

Epoch 10/250
1536/2000 [======================>.......] - ETA: 0s - loss: 0.6462 - accuracy: 0.5417
Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
2000/2000 [==============================] - 4s 2ms/sample - loss: 0.6802 - accuracy: 0.5360 - val_loss: 0.7125 - val_accuracy: 0.4973
Epoch 11/250
2000/2000 [==============================] - 4s 2ms/sample - loss: 0.6562 - accuracy: 0.5440 - val_loss: 0.7134 - val_accuracy: 0.4981
Epoch 12/250
2000/2000 [==============================] - 4s 2ms/sample - loss: 0.6568 - accuracy: 0.5410 - val_loss: 0.7075 - val_accuracy: 0.4972
Epoch 13/250
2000/2000 [==============================] - 4s 2ms/sample - loss: 0.6548 - accuracy: 0.5375 - val_loss: 0.7018 - val_accuracy: 0.4974
Epoch 14/250
2000/2000 [==============================] - 4s 2ms/sample - loss: 0.6528 - accuracy: 0.5285 - val_loss: 0.7007 - val_accuracy: 0.4919
Epoch 15/250
2000/2000 [==============================] - 4s 2ms/sample - loss: 0.6508 - accu

Epoch 00019: early stopping
Train on 4000 samples, validate on 10000 samples
Epoch 1/250
4000/4000 [==============================] - 10s 2ms/sample - loss: 0.6954 - accuracy: 0.4980 - val_loss: 0.6943 - val_accuracy: 0.5000
Epoch 2/250
4000/4000 [==============================] - 5s 1ms/sample - loss: 0.6932 - accuracy: 0.5040 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 3/250
4000/4000 [==============================] - 5s 1ms/sample - loss: 0.6924 - accuracy: 0.5030 - val_loss: 0.6943 - val_accuracy: 0.5057
Epoch 4/250
4000/4000 [==============================] - 5s 1ms/sample - loss: 0.6857 - accuracy: 0.5288 - val_loss: 0.7499 - val_accuracy: 0.5004
Epoch 5/250
4000/4000 [==============================] - 5s 1ms/sample - loss: 0.6703 - accuracy: 0.5255 - val_loss: 0.7055 - val_accuracy: 0.5044
Epoch 6/250
4000/4000 [==============================] - 5s 1ms/sample - loss: 0.6604 - accuracy: 0.5293 - val_loss: 0.7198 - val_accuracy: 0.4961
Epoch 7/250
4000/4000 [=================

Epoch 8/250
12000/12000 [==============================] - 12s 960us/sample - loss: 0.6394 - accuracy: 0.5378 - val_loss: 0.7504 - val_accuracy: 0.5076
Epoch 9/250
12000/12000 [==============================] - 13s 1ms/sample - loss: 0.6407 - accuracy: 0.5437 - val_loss: 0.9216 - val_accuracy: 0.4801
Epoch 10/250
11776/12000 [============================>.] - ETA: 0s - loss: 0.9164 - accuracy: 0.5131
Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
12000/12000 [==============================] - 13s 1ms/sample - loss: 0.9135 - accuracy: 0.5141 - val_loss: 0.7901 - val_accuracy: 0.4999
Epoch 11/250
12000/12000 [==============================] - 12s 992us/sample - loss: 0.7837 - accuracy: 0.5002 - val_loss: 0.7056 - val_accuracy: 0.5051
Epoch 12/250
12000/12000 [==============================] - 13s 1ms/sample - loss: 0.7327 - accuracy: 0.4955 - val_loss: 0.6932 - val_accuracy: 0.4956
Epoch 13/250
12000/12000 [==============================] - 12s 963us/samp

Epoch 2/250
16000/16000 [==============================] - 16s 985us/sample - loss: 0.6913 - accuracy: 0.5132 - val_loss: 0.6971 - val_accuracy: 0.5027
Epoch 3/250
16000/16000 [==============================] - 17s 1ms/sample - loss: 0.6696 - accuracy: 0.5326 - val_loss: 0.7114 - val_accuracy: 0.5027
Epoch 4/250
16000/16000 [==============================] - 17s 1ms/sample - loss: 0.6476 - accuracy: 0.5369 - val_loss: 0.7372 - val_accuracy: 0.5067
Epoch 5/250
16000/16000 [==============================] - 16s 1ms/sample - loss: 0.6521 - accuracy: 0.5515 - val_loss: 0.7498 - val_accuracy: 0.5079
Epoch 6/250
16000/16000 [==============================] - 16s 1ms/sample - loss: 0.6403 - accuracy: 0.5390 - val_loss: 0.7638 - val_accuracy: 0.5057
Epoch 7/250
16000/16000 [==============================] - 16s 995us/sample - loss: 0.6476 - accuracy: 0.5447 - val_loss: 0.7396 - val_accuracy: 0.5096
Epoch 8/250
16000/16000 [==============================] - 17s 1ms/sample - loss: 0.6414 - accur

20000/20000 [==============================] - 18s 889us/sample - loss: 0.1683 - accuracy: 0.9471 - val_loss: 0.4329 - val_accuracy: 0.8462
Epoch 19/250
20000/20000 [==============================] - 18s 880us/sample - loss: 0.1453 - accuracy: 0.9556 - val_loss: 0.4544 - val_accuracy: 0.8503
Epoch 20/250
20000/20000 [==============================] - 18s 880us/sample - loss: 0.1317 - accuracy: 0.9617 - val_loss: 0.4823 - val_accuracy: 0.8459
Epoch 21/250
20000/20000 [==============================] - 18s 879us/sample - loss: 0.1217 - accuracy: 0.9662 - val_loss: 0.5016 - val_accuracy: 0.8400
Epoch 22/250
20000/20000 [==============================] - 18s 878us/sample - loss: 0.1039 - accuracy: 0.9721 - val_loss: 0.4995 - val_accuracy: 0.8529
Epoch 23/250
20000/20000 [==============================] - 18s 883us/sample - loss: 0.0906 - accuracy: 0.9788 - val_loss: 0.4953 - val_accuracy: 0.8564
Epoch 24/250
19968/20000 [============================>.] - ETA: 0s - loss: 0.0799 - accuracy: 

24000/24000 [==============================] - 21s 895us/sample - loss: 0.6369 - accuracy: 0.5394 - val_loss: 0.8109 - val_accuracy: 0.5048
Epoch 17/250
23552/24000 [============================>.] - ETA: 0s - loss: 0.6353 - accuracy: 0.5487
Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.0001999999862164259.
24000/24000 [==============================] - 21s 881us/sample - loss: 0.6354 - accuracy: 0.5485 - val_loss: 0.8222 - val_accuracy: 0.5058
Epoch 00017: early stopping
Train on 24000 samples, validate on 10000 samples
Epoch 1/250
24000/24000 [==============================] - 34s 1ms/sample - loss: 0.6944 - accuracy: 0.5048 - val_loss: 0.6932 - val_accuracy: 0.5058
Epoch 2/250
24000/24000 [==============================] - 22s 917us/sample - loss: 0.6902 - accuracy: 0.5127 - val_loss: 0.6931 - val_accuracy: 0.5077
Epoch 3/250
24000/24000 [==============================] - 22s 904us/sample - loss: 0.6691 - accuracy: 0.5257 - val_loss: 0.7042 - val_accuracy: 0.5102
Epoch 

28000/28000 [==============================] - 25s 903us/sample - loss: 0.3769 - accuracy: 0.8458 - val_loss: 0.4734 - val_accuracy: 0.7985
Epoch 15/250
28000/28000 [==============================] - 25s 907us/sample - loss: 0.3306 - accuracy: 0.8721 - val_loss: 0.4496 - val_accuracy: 0.8177
Epoch 16/250
28000/28000 [==============================] - 25s 910us/sample - loss: 0.2951 - accuracy: 0.8905 - val_loss: 0.4380 - val_accuracy: 0.8368
Epoch 17/250
28000/28000 [==============================] - 25s 911us/sample - loss: 0.2592 - accuracy: 0.9092 - val_loss: 0.4154 - val_accuracy: 0.8401
Epoch 18/250
28000/28000 [==============================] - 25s 904us/sample - loss: 0.2150 - accuracy: 0.9294 - val_loss: 0.4302 - val_accuracy: 0.8516
Epoch 19/250
28000/28000 [==============================] - 25s 909us/sample - loss: 0.2043 - accuracy: 0.9339 - val_loss: 0.4375 - val_accuracy: 0.8552
Epoch 20/250
28000/28000 [==============================] - 25s 904us/sample - loss: 0.1716 - a

Epoch 25/250
32000/32000 [==============================] - 28s 867us/sample - loss: 0.0461 - accuracy: 0.9915 - val_loss: 0.5134 - val_accuracy: 0.8747
Epoch 26/250
32000/32000 [==============================] - 28s 861us/sample - loss: 0.0448 - accuracy: 0.9918 - val_loss: 0.5203 - val_accuracy: 0.8740
Epoch 27/250
32000/32000 [==============================] - 28s 860us/sample - loss: 0.0439 - accuracy: 0.9924 - val_loss: 0.5254 - val_accuracy: 0.8727
Epoch 28/250
32000/32000 [==============================] - 27s 851us/sample - loss: 0.0427 - accuracy: 0.9927 - val_loss: 0.5373 - val_accuracy: 0.8729
Epoch 29/250
31744/32000 [============================>.] - ETA: 0s - loss: 0.0428 - accuracy: 0.9928
Epoch 00029: ReduceLROnPlateau reducing learning rate to 0.0001999999862164259.
32000/32000 [==============================] - 27s 853us/sample - loss: 0.0425 - accuracy: 0.9928 - val_loss: 0.5322 - val_accuracy: 0.8723
Epoch 00029: early stopping
Training using 0.9 of dataset.
Train o

In [91]:
# Average out the validation accuracy of all three iterations, for each subset of data we used to train
# and store it in a dictionary. Do this for all subsets we have (0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9)

new_dict = {}
for key, item in RNN_noaug_hist.items():
    avg_list = []
    for history in item:
        print(history.history['val_accuracy'])
        avg_list.append(history.history['val_accuracy'])
    print("========================================================")
    avg_list = avgNestedLists(avg_list)
    new_dict[key] = avg_list
    print(avg_list)
    print()

[0.4999, 0.5, 0.5, 0.5031, 0.5041, 0.5, 0.5, 0.505, 0.5043, 0.5048, 0.5, 0.5002, 0.5053, 0.5052, 0.5056, 0.5055, 0.5051, 0.5005, 0.5002, 0.5003, 0.5002, 0.5004, 0.5003, 0.5003]
[0.5, 0.5, 0.5, 0.5, 0.5054, 0.5023, 0.5008, 0.5032, 0.4985, 0.4984, 0.4989, 0.498, 0.5037, 0.5038, 0.5036, 0.5039, 0.5038, 0.5038, 0.5038, 0.5037, 0.5036, 0.5037, 0.5038, 0.5037, 0.5039, 0.5039, 0.5037, 0.5035, 0.5036]
[0.5, 0.4999, 0.5, 0.5, 0.5, 0.5043, 0.5021, 0.501, 0.5023, 0.5033, 0.4987, 0.4994, 0.5026, 0.4978, 0.4983, 0.4989, 0.5044, 0.5038, 0.4993, 0.5037, 0.5028, 0.4978]
[0.49996665, 0.49996665, 0.5, 0.5010333, 0.5031667, 0.5022, 0.50096667, 0.5030666, 0.5017, 0.5021667, 0.4992, 0.4992, 0.5038667, 0.5022667, 0.5025, 0.50276667, 0.50443333, 0.50270003, 0.5011, 0.50256664, 0.50219995, 0.50063336, 0.50205, 0.502, 0.5039, 0.5039, 0.5037, 0.5035, 0.5036]

[0.5, 0.5031, 0.5, 0.5052, 0.5026, 0.5014, 0.4992, 0.5009, 0.4948, 0.5016, 0.4965, 0.4959, 0.4992, 0.5011, 0.5006, 0.4979, 0.4943, 0.4971, 0.4996, 0.4974]

In [96]:
# Write the results into csv files with the name being the model name and the subset size

for title, item in new_dict.items():
    temp_dict = {title:item}
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(temp_dict)

    # save to csv:
    hist_csv_file = 'RNN_NoAug_History_' + str(title) + '.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)